In [1]:
import os
import re
import yaml
import glob

from math import ceil

import numpy as np
import pandas as pd

from matplotlib import cm

import zarr
import napari
import tifffile
import dask.array as da

from utils.utility_functions import single_channel_pyramid

In [2]:
# Paths and input

sample = 'CRC-097'

tif_path = os.path.join(os.getcwd(), f'input/{sample}_image.ome.tif')
seg_path = os.path.join(os.getcwd(), f'input/{sample}_seg_outlines.ome.tif')

# Read single-cell sample for VAE analysis
main = pd.read_csv(os.path.join(os.getcwd(), 'input/main.csv'))
main = main[main['VAE20'] != -1]

# Import config.yml
with open(os.path.join(os.getcwd(), 'input/CRC-97_cylinter_config.yml')) as f:
    config = yaml.safe_load(f)
markers_to_exclude = config['markersToExclude']  # channelExclusionsClustering

# Import markers.csv
markers = pd.read_csv(os.path.join(os.getcwd(), 'input/CRC-097_mcmicro_markers.csv'))

# Get first name of first DNA channel
dna1 = markers['marker_name'][markers['channel_number'] == 1][0]
dna_moniker = str(re.search(r'[^\W\d]+', dna1).group())

# Import image contrast settings
with open(os.path.join(os.getcwd(), 'input/CRC-097_cylinter_contrast_limits.yml')) as f:
    contrast_limits = yaml.safe_load(f)

# The parquet file at the path below is being read because "main.csv" 
# uses trimmed marker channel names as column headers that differ from the raw channel names used 
# in the markers.csv file, which is itself used to index channels in the OME-TIFF image.
for_channels = pd.read_parquet(
    os.path.join(os.getcwd(), 'input/CRC-097_clean_cylinter_clustering_3d_leiden.parquet')
)

# Isolate antibodies of interest
abx_channels = [
    i for i in for_channels.columns if 'nucleiRingMask' in i
    if 'Hoechst' not in i
    if i not in markers_to_exclude]

# out = os.path.join(os.getcwd(), f'output/VAE20_metaclusters')
# if not os.path.exists(out):
#     os.makedirs(out)

In [3]:
# Group VAE20 clusters into metaclusters
transformed_epithelial = [
    29, 31, 13, 51, 48, 56, 54, 24, 55, 57, 52, 53, 25, 47, 27, 49, 50, 46, 15,
    12, 42, 10, 32, 40, 41, 14, 11
]

normal_epithelial = [1]

rbm = [2]

lymph_follicles = [3, 4, 5, 6]

granulation = [7, 44, 45, 43]

cd45_bright = [21, 22]

stroma = [8, 16, 26, 20, 28, 30, 37, 36, 33]

interstitium = [35, 18, 19, 9, 17, 39, 38, 34, 23]

artifact = [0]

In [4]:
for set in [
    transformed_epithelial, normal_epithelial, rbm, lymph_follicles,
    granulation, cd45_bright, stroma, artifact]:
    
    viewer = napari.Viewer()
    
    # Read DNA1 channel
    dna, min, max = single_channel_pyramid(glob.glob(tif_path)[0], channel=0)

    # Add DNA1 channel to Napari image viewer
    viewer.add_image(
        dna, rgb=False, blending='opaque', colormap='gray', visible=True,
        opacity=0.75, name='DNA1', contrast_limits=(min, max)
    )

    # Loop over antibodies of interest and add them to Napari image viewer
    for ch in abx_channels:
        ch = ch.rsplit('_', 1)[0]
        channel_number = markers['channel_number'][markers['marker_name'] == ch]
        img, min, max = single_channel_pyramid(
            glob.glob(tif_path)[0], channel=(channel_number.item() - 1)
        )

        viewer.add_image(
            img, rgb=False, blending='additive', colormap='green',
            visible=False, name=ch, contrast_limits=(min, max)
        )

    # Apply previously defined contrast limits
    for ch in abx_channels:
        ch = ch.rsplit('_', 1)[0]
        viewer.layers[ch].contrast_limits = (
            contrast_limits[ch][0], contrast_limits[ch][1])

    # Add centroids of cells for each VAE cluster to Napari image viewer
    num_colors = len(list(cm.tab10.colors))
    num_clusters = len(set)
    palette_multiplier = ceil(num_clusters / num_colors)
    colors = list(cm.tab10.colors) * palette_multiplier
    colors = colors[0:num_clusters]
    colors.reverse()

    for c, cluster in zip(colors, sorted(set, reverse=True)):
        centroids = main[['Y_centroid', 'X_centroid']][main['VAE20'] == cluster]
        viewer.add_points(
            centroids, name=f'VAE20_{cluster}',
            face_color=np.array(c),
            edge_color=np.array(c),
            edge_width=0.0, size=50.0, opacity=1.0, blending='translucent',
            visible=True
        )

    # Read segmentation outlines
    seg, min, max = single_channel_pyramid(glob.glob(seg_path)[0], channel=0)
    viewer.add_image(
        seg, rgb=False, blending='additive', colormap='gray', visible=False,
        name='segmentation', opacity=0.3, contrast_limits=(min, max)
    )
    
    # Run Napari image viewer
    viewer.scale_bar.visible = True
    viewer.scale_bar.unit = 'um'

    napari.run()